# 几个题目

## 1.What is autoencoder?
+ 自编码器，简称AE，结合神经网络构成一套具有自动编码功能的模型
+ 编码的本质是将信息从一种格式/形式转为另一种格式/形式的过程，目的是获取转换后格式/形式的优良特性。比如节约空间，方便相似度计算。
+ AutoEncoder是无监督学习，即学习过程不需要使用样本的label，本质上是把样本的输入同时作为神经网络的输入和输出，通过最小化重构误差希望学习到样本的抽象特征表示Vector。
+ 通过AutoEncoder可以对高维数据进行高效的特征提取和表示

## 2.What are the differences between greedy search and beam search?
greedy search：使用softmax取出概率最大的一个结果。</br>
beam search：beam是束宽度，每一步从概率排序的前beam个结果中随机取出一个作为最终结果

## 3.What is the intuition of attention mechanism?
+ 2015年提出，让Encoder编码出的c向量跟Decoder解码过程中的每一个输出进行加权运算，在Decoder的每一个过程中调整权重取到不一样的c向量。</br>
+ Attention机制就是让Encoder编码出来的向量根据Decoder要解码的内容动态变化，类似人的视觉在看某一个物体的焦点，也就是对于重要部分设置更高权重
+ 假设Encoder每个隐藏状态为$h_j$，序列长度为T，那么在第i个时刻c向量计算为</br>
</br>$c_i = \sum^{T}_{j=1}{a_{ij}}{h_j}$ </br>
</br>其中，</br>
</br>$a_{ij} = softmax(e(s_{i-1},h_j))$</br>
</br>是Attention分数,j=1,2,...,T时刻的一个概率分布，用softmax进行计算，$s_{i-1}$ 表示Decoder在i-1时刻的状态，e为距离函数，比如使用内积。</br>
</br>下一时刻的隐藏状态</br>
</br>$s_i = f(s_{i-1}, y_{i-1}, c_i)$</br>

# 中英文自动翻译模型的构建（使用encoder-decoder模型）

In [3]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [4]:
# 设备设置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SOS_token = 0
EOS_token = 1

In [5]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {0:"<SOS>", 1:"<EOS>", -1:"<UNK>"}
        self.idx = 2 # 当前长度
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def add_sentence(self, sentence):
        for word in sentence.split():
            self.add_word(word)

    # 得到某个词的id
    def __call__(self, word):
        if not word in self.word2idx:
            return -1
        return self.word2idx[word]
    
    # vocabulary的容量
    def __len__(self):
        return self.idx

In [6]:
# encoder-decoder模型的搭建
# 1，模型训练    
# 2，模型推理     
# 3，模型预测，展示结果
class EncoderRNN(nn.Module):
    # 在构造函数内定义Embedding层和GRU层
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input_size代表输入语言的所有单词的数量，hidden_size是GRU网络的隐藏层节点数
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    # 前向传播
    def forward(self, input, hidden):
        # seq_len=1, batch=1
        embedded = self.embedding(input).view(1, 1, self.hidden_size)
        output, hidden = self.gru(embedded, hidden)
        return hidden
        
    # 最终执行函数
    def sample(self, seq_list):
        word_inds = torch.LongTensor(seq_list).to(device)
        h = self.initHidden()
        for word_tensor in word_inds:
            h = self(word_tensor, h)
        return h
        
    # 初始化第一层h0，随机生成一个
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.maxlen = 10
        # output_size 是输出语言的所有单词的数量，hidden_size是GRU网络的隐藏层节点数
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        # Linear作用是将前面的GRU的输出结果变成目标语言的单词的长度
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    # 前向传播
    def forward(self, seq_input, hidden):
        output = self.embedding(seq_input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    # 最终执行函数
    def sample(self, pre_hidden):
        inputs = torch.tensor([SOS_token], device=device)
        hidden = pre_hidden
        res = [SOS_token]
        # 循环编码
        for i in range(self.maxlen):
            output, hidden = self(inputs, hidden)
            # 需要获取最大索引作为生成单词的id
            topv, topi = output.topk(1) # value, index
            # 遇到句子结束符，解码结束
            if topi.item() == EOS_token:
                res.append(EOS_token)
                break
            else:
                res.append(topi.item())
            # 将生成的单词作为下一时刻的输入，squeeze()去掉维度为1的维度，detach保证梯度不传导   
            inputs = topi.squeeze().detach()
        return res

In [8]:
# 处理句子，将句子转换成Tensor
def sentence2tensor(lang, sentence):
    indexes = [lang(word) for word in sentence.split()]
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# 将(input, target)的pair都转换成Tensor
def pair2tensor(pair):
    input_tensor = sentence2tensor(lan1, pair[0])
    target_tensor = sentence2tensor(lan2, pair[1])
    return (input_tensor, target_tensor)

In [9]:
# 定义句子和Vocabulary类
lan1 = Vocabulary()
lan2 = Vocabulary()
# 准备数据集
data = [['Hi.','嗨。'],
    ['Hi.', '你 好。'],
    ['Run.', '跑。'],
    ['Wait!', '等等！'],
    ['Hello!','你好。'],
    ['I try.','让 我 来。'],
    ['I won!','我 赢了！'],
    ['I am ok.', '我 很好。']
]

for i, j in data:
    lan1.add_sentence(i)
    lan2.add_sentence(j)
print(len(lan1))
print(len(lan2))

11
13


In [10]:
import random
# 定义参数
learning_rate = 0.001
hidden_size = 256

encoder = EncoderRNN(len(lan1), hidden_size).to(device)
decoder = DecoderRNN(hidden_size, len(lan2)).to(device)
# 网络参数
params = list(encoder.parameters()) + list(decoder.parameters())
# 定义优化器
optimizer = optim.Adam(params, lr=learning_rate)
loss = 0
criterion = nn.NLLLoss() # Negative log likelihood Loss
# 一共训练多少轮
turns = 2000
print_every = 200
print_loss_total = 0
# 创建随机数据
training_pairs = [pair2tensor(random.choice(data)) for pair in range(turns)]

In [11]:
# 训练过程
for turn in range(turns):
    optimizer.zero_grad()
    loss = 0
    x, y = training_pairs[turn]
    input_length = x.size(0)
    target_length = y.size(0)
    # 初始化Encoder中的h0
    h = encoder.initHidden()
    # 对input进行Encoder
    for i in range(input_length):
        h = encoder(x[i], h)
    # Decoder的一个input
    decoder_input = torch.LongTensor([SOS_token]).to(device)
    
    for i in range(target_length):
        decoder_output, h = decoder(decoder_input, h)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        loss = loss + criterion(decoder_output, y[i])
        if decoder_input.item() == EOS_token:
            break
    print_loss_total = print_loss_total + loss.item() / target_length
    if (turn+1) % print_every == 0:
        print('loss:{:.4f}'.format(print_loss_total/print_every))
        print_loss_total = 0
        
    #optimizer.zero_grad()
    loss.backward()
    optimizer.step()

loss:0.4897
loss:0.1753
loss:0.1926
loss:0.1533
loss:0.1449
loss:0.1563
loss:0.1570
loss:0.1473
loss:0.1426
loss:0.1599


In [12]:
# 测试
def translate(s):
    t = [lan1(i) for i in s.split()]
    t.append(EOS_token)
    f = encoder.sample(t)
    s = decoder.sample(f)
    r = [lan2.idx2word[i] for i in s]
    return ' '.join(r) 

translate('Hi.')

'<SOS> 你 <EOS>'